# 第七周练习题 Multicollinearity 
## 背景描述  
McDonald和Schwing(1973)提出了一项研究，将总死亡率与气候、社会经济和污染变量联系起来，本研究选择了 15 个自变量列于下表。因变量是以上原因在年龄调整后的总死亡率。我们不对该研究的流行病学方面作评论，而仅仅作为变量选择的说明性例子使用这些数据。  
由此我们构造了 60 个观测的 15 个变量，具体请见下表：

## 数据描述

|     变量名      |            变量含义            |      变量类型       |  变量取值范围  |
| :-------------: | :----------------------------: | :-----------------: | :------------: |
|  （自变量1）X1  |       年平均降水量(英寸)       | continuous variable | $\mathbb{R}^+$ |
|  （自变量2）X2  |      一月平均气温(华氏度)      | continuous variable | $\mathbb{R}^+$ |
|  （自变量3）X3  |      七月平均气温(华氏度)      | continuous variable | $\mathbb{R}^+$ |
|  （自变量4）X4  |      占65岁以上人口的比例      | continuous variable | $\mathbb{R}^+$ |
|  （自变量5）X5  |          每个家庭人口          | continuous variable | $\mathbb{R}^+$ |
|  （自变量6）X6  |         完成的平均学制         | continuous variable | $\mathbb{R}^+$ |
|  （自变量7）X7  |     健全的住房单位的百分比     | continuous variable | $\mathbb{R}^+$ |
|  （自变量8）X8  |         每平方英里人口         | continuous variable | $\mathbb{R}^+$ |
|  （自变量9）X9  |       占非白人人口的比例       | continuous variable | $\mathbb{R}^+$ |
| （自变量10）X10 |        白领工作的就业率        | continuous variable | $\mathbb{R}^+$ |
| （自变量11）X11 | 收入在3000美元以下家庭的百分比 | continuous variable | $\mathbb{R}^+$ |
| （自变量12）X12 |    碳氢化合物的相对污染潜力    | continuous variable | $\mathbb{R}^+$ |
| （自变量13）X13 |     氮氧化物的相对污染潜力     | continuous variable | $\mathbb{R}^+$ |
| （自变量14）X14 |     二氧化硫的相对污染潜力     | continuous variable | $\mathbb{R}^+$ |
| （自变量15）X15 |         相对湿度百分比         | continuous variable | $\mathbb{R}^+$ |
|   （因变量）Y   |  所有原因经年龄调整的总死亡率  | continuous variable | $\mathbb{R}^+$ |

## 问题 
注：这里使用 \alpha=0.05 的显著性水平

1. 对数据建立多元线性回归模型.
2. 判断这些特征之间是否存在多重共线性，并给出判断依据.
3. 以 BIC 为指标，对模型进行逐步回归做变量选择，判断变量选择后是否存在多重共线性。  

## 解决方案

## Q1：对数据建立多元线性回归模型.

对因变量“所有原因经年龄调整的总死亡率”建立多元线性回归模型，考虑这15个特征对因变量的影响。

In [1]:
# Import standard packages
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# Import additional packages
from itertools import combinations
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

alpha = 0.05
p = 15
n = 60

x = pd.read_csv('D:\\mycode\\大三上\\可视化code\\统计学习方法\\Multicollinearity\\homework\\Air_Pollution.csv')
#加第0列，第0列全为1
x.insert(0, '0', np.ones(len(x))) 
data = x.values * 1
df = pd.DataFrame(data)
print(df.head())

#X矩阵
X = data[:,0:p+1]
Y = data[:,-1]

#一些准备工作 数据预处理，对自变量 X 进行标准化

# 求自变量 X 的标准差
X_mean = []
for i in range(p):
    X_mean.append(np.mean(X[:, i+1])) 
X_sigma = []
for i in range(p):
    X_sigma.append(sum((X[:, i+1] - X_mean[i]) ** 2))  

# 对自变量 X 标准化(截距项不用标准化)
X_std = X * 1.0
X_std[:,1:p+1] = (X[:,1:p+1] - X_mean) / np.sqrt(X_sigma)

    0     1     2     3     4     5     6     7       8     9     10    11  \
0  1.0  36.0  27.0  71.0   8.1  3.34  11.4  81.5  3243.0   8.8  42.6  11.7   
1  1.0  35.0  23.0  72.0  11.1  3.14  11.0  78.8  4281.0   3.5  50.7  14.4   
2  1.0  44.0  29.0  74.0  10.4  3.21   9.8  81.6  4260.0   0.8  39.4  12.4   
3  1.0  47.0  45.0  79.0   6.5  3.41  11.1  77.5  3125.0  27.1  50.2  20.6   
4  1.0  43.0  35.0  77.0   7.6  3.44   9.6  84.6  6441.0  24.4  43.7  14.3   

     12    13     14    15       16  
0  21.0  15.0   59.0  59.0   921.87  
1   8.0  10.0   39.0  57.0   997.88  
2   6.0   6.0   33.0  54.0   962.35  
3  18.0   8.0   24.0  56.0   982.29  
4  43.0  38.0  206.0  55.0  1071.29  


**做多元线性回归分析**

In [2]:
# multiple linear regression
model_std = sm.OLS(Y, X_std).fit()
Y_std_hat = model_std.fittedvalues
model_std.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     9.542
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           2.19e-09
Time:                        12:46:51   Log-Likelihood:                -289.03
No. Observations:                  60   AIC:                             610.1
Df Residuals:                      44   BIC:                             643.6
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.509    208.538      0.000     931.271     949.446
x1           146.1294     70.845      2.063      0.045       3.351     288.908
x2          -151.3453     86.575     -1.748      0.087    -325.826      23.135
x3          -113.4333     69.576     -1.630      0.110    -253.654      26.787
x4          -101.9781     95.465     -1.068      0.291    -294.376      90.419
x5          -110.9860     72.494     -1.531      0.133    -257.088      35.116
x6          -111.3974     77.006     -1.447      0.155    -266.593      43.798
x7           -25.7135     69.812     -0.368      0.714    -166.410     114.983
x8            40.2161     44.979      0.894      0.376     -50.434     130.866
x9           305.5909     90.946      3.360      0.002     122.300     488.881
x10           -6.6313     58.879     -0.113      0.911    -125.295     112.032
x11           -5.3495    103.126     -0.052      0.959    -213.187     202.488
x12         -474.8764    346.905     -1.369      0.178   -1174.016     224.264
x13          476.9329    357.884      1.333      0.190    -244.334    1198.200
x14           42.0025     71.829      0.585      0.562    -102.759     186.764
x15            4.4026     48.236      0.091      0.928     -92.810     101.615
==============================================================================
Omnibus:                        2.991   Durbin-Watson:                   2.129
Prob(Omnibus):                  0.224   Jarque-Bera (JB):                2.088
Skew:                           0.359   Prob(JB):                        0.352
Kurtosis:                       3.566   Cond. No.                         111.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Q2：判断这些特征之间是否存在多重共线性，并给出判断依据.

【法1】  直观判定法

In [3]:
# 相关系数
r = df.corr()
r

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.000000,0.092208,0.503273,0.101113,0.263444,-0.490425,-0.490759,-0.003515,0.413204,-0.297291,0.506585,-0.531760,-0.487321,-0.106924,-0.077343,0.509500
2,NaN,0.092208,1.000000,0.346282,-0.398099,-0.209212,0.116284,0.014852,-0.100051,0.453774,0.237992,0.565314,0.350809,0.321014,-0.107810,0.067872,-0.030022
3,NaN,0.503273,0.346282,1.000000,-0.434040,0.262280,-0.238544,-0.415032,-0.060994,0.575309,-0.021412,0.619308,-0.356494,-0.337668,-0.099348,-0.452809,0.277018
4,NaN,0.101113,-0.398099,-0.434040,1.000000,-0.509087,-0.138862,0.065010,0.161991,-0.637821,-0.117715,-0.309771,-0.020486,-0.002082,0.017248,0.112426,-0.174593
5,NaN,0.263444,-0.209212,0.262280,-0.509087,1.000000,-0.395075,-0.410590,-0.184332,0.419410,-0.425723,0.259904,-0.388205,-0.358429,-0.004084,-0.135740,0.357307
6,NaN,-0.490425,0.116284,-0.238544,-0.138862,-0.395075,1.000000,0.552237,-0.243883,-0.208774,0.703196,-0.403338,0.286835,0.224402,-0.234346,0.176491,-0.510988
7,NaN,-0.490759,0.014852,-0.415032,0.065010,-0.410590,0.552237,1.000000,0.181881,-0.410334,0.338745,-0.680680,0.386767,0.348250,0.117952,0.121901,-0.426821
8,NaN,-0.003515,-0.100051,-0.060994,0.161991,-0.184332,-0.243883,0.181881,1.000000,-0.005678,-0.031765,-0.162945,0.120282,0.165312,0.432086,-0.124976,0.265503
9,NaN,0.413204,0.453774,0.575309,-0.637821,0.419410,-0.208774,-0.410334,-0.005678,1.000000,-0.004387,0.704915,-0.025865,0.018385,0.159293,-0.117957,0.643742


1. 当某自变量与因变量之间的简单相关系数绝对值很大，在回归方程中没有通过显著性检验时，便可初步判断存在多重共线性。

In [4]:
r_xy = np.array(r.iloc[1:p+1][p+1])
print('因变量和每个自变量之间的相关系数: \n', r_xy)

judge_xy = True
for i in range(p):
    if (abs(r_xy[i]) >= 0.5) & (model_std.pvalues[i+1] >= alpha):
        judge_xy = False
        print('自变量 %d 与因变量之间的简单相关系数为: %.4f, tPal: %.4f.' % (i+1, r_xy[i], model_std.pvalues[i+1]))
        
if judge_xy:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n存在多重共线性。')

因变量和每个自变量之间的相关系数: 
 [ 0.50949986 -0.03002188  0.27701762 -0.17459291  0.35730691 -0.51098849
 -0.42682123  0.2655034   0.64374176 -0.28480459  0.41049037 -0.17724211
 -0.07738243  0.42589789 -0.08850055]
自变量 6 与因变量之间的简单相关系数为: -0.5110, tPal: 0.1551.

存在多重共线性。


2. 当自变量间的相关系数较大时，也可以初步判断出现多重共线性。这里设阀值为0.6

In [5]:
judge_xx = True
for (i, j) in combinations(range(1, p+2), 2):
    if(r.iloc[i][j] >= 0.6):
        judge_xx = False
        print('变量(%d,%d)之间相关系数较大，为：%.4f'% (i, j, r.iloc[i][j]))
        
if judge_xx:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n自变量之间存在多重共线性。')

变量(3,11)之间相关系数较大，为：0.6193
变量(6,10)之间相关系数较大，为：0.7032
变量(9,11)之间相关系数较大，为：0.7049
变量(9,16)之间相关系数较大，为：0.6437
变量(12,13)之间相关系数较大，为：0.9838

自变量之间存在多重共线性。


结论：直观判定法认为这些特征之间存多重共线性。

【法2】方差扩大因子法  
1. 计算自变量 $x_j$ 的方差扩大因子 $\mathsf{VIF_j}$，$j=1,\cdots,p$.

In [6]:
# 秋方差扩大因子有两种方法

# 法1：
c = np.dot(X_std.T, X_std)
C = np.linalg.inv(c)  
C_list = []
for i in range(p):
    C_list.append(C[i + 1][i + 1])

# 法2：
vif = [variance_inflation_factor(X_std[:,1:p + 1], i) for i in range(p)]

print('C主对角线元素  方差扩大因子：')
for i in range(p):
    print('%d. %.4f        %.4f' % (i+1, C_list[i], vif[i]))

C主对角线元素  方差扩大因子：
1. 4.1139        4.1139
2. 6.1436        6.1436
3. 3.9678        3.9678
4. 7.4700        7.4700
5. 4.3076        4.3076
6. 4.8605        4.8605
7. 3.9948        3.9948
8. 1.6583        1.6583
9. 6.7796        6.7796
10. 2.8416        2.8416
11. 8.7171        8.7171
12. 98.6399        98.6399
13. 104.9824        104.9824
14. 4.2289        4.2289
15. 1.9071        1.9071


2. 通过 $\mathsf{VIF_j}$ 的大小判断自变量之间是否存在多重共线性.  
    VIF值大于10说明共线性很严重

    VIF值在5以下不需要处理

    VIF介于5~10之间视情况而定。

In [7]:
thres_vif = 5
flag = False
for i in range(p):
    if vif[i] >= thres_vif:
        flag = True
        print('自变量 x%d 与其余自变量之间存在多重共线性，其中VIF值为：%.4f' % (i + 1, vif[i]))

if flag==False:
    print('这种判断方式表明变量间不存在多重共线性.')
print('\n这种判断方式表明变量间存在多重共线性.')

自变量 x2 与其余自变量之间存在多重共线性，其中VIF值为：6.1436
自变量 x4 与其余自变量之间存在多重共线性，其中VIF值为：7.4700
自变量 x9 与其余自变量之间存在多重共线性，其中VIF值为：6.7796
自变量 x11 与其余自变量之间存在多重共线性，其中VIF值为：8.7171
自变量 x12 与其余自变量之间存在多重共线性，其中VIF值为：98.6399
自变量 x13 与其余自变量之间存在多重共线性，其中VIF值为：104.9824

这种判断方式表明变量间存在多重共线性.


结论：由方法2方差扩大因子可以判断存在多重共线性。

【法3】特征值判定法  
1. 计算自变量 $x_j$ 的条件数 $\kappa_j = \sqrt{\frac{\lambda_1}{\lambda_j}}$，$j=1,\cdots,p$.

In [8]:
 # 计算相关系数矩阵
corr = np.corrcoef(X_std[:,1:p+1], rowvar = 0)
# 特征值 & 特征向量
w, v = np.linalg.eig(corr) 

kappa = []
for i in range(p):
    kappa.append(np.sqrt(max(w) / w[i]))
    print('特征值%d: %.4f, kappa%d: %.4f' %(i + 1, w[i], i + 1, kappa[i]))

特征值1: 4.5284, kappa1: 1.0000
特征值2: 2.7548, kappa2: 1.2821
特征值3: 2.0545, kappa3: 1.4846
特征值4: 1.3484, kappa4: 1.8326
特征值5: 1.2232, kappa5: 1.9241
特征值6: 0.9604, kappa6: 2.1714
特征值7: 0.6127, kappa7: 2.7185
特征值8: 0.4720, kappa8: 3.0974
特征值9: 0.3709, kappa9: 3.4944
特征值10: 0.0049, kappa10: 30.5051
特征值11: 0.0460, kappa11: 9.9176
特征值12: 0.2164, kappa12: 4.5746
特征值13: 0.1664, kappa13: 5.2175
特征值14: 0.1140, kappa14: 6.3030
特征值15: 0.1270, kappa15: 5.9712


2. 通过 $\kappa_p$ 的大小判断自变量之间是否存在多重共线性以及多重共线性的严重程度.  
记 $\kappa=\lambda_{max}/ \lambda_{min}$.

一般若 $\kappa<100$，则认为多重共线性的程度很小

若 $100<=\kappa<=1000$，则认为存在一般程度上的多重共线性

若 $\kappa>1000$，则认为存在严重的多重共线性

$\kappa >= c_{\kappa}$时，自变量之间存在多重共线性，$c_{\kappa}$常见取值为10，100，1000.

In [9]:
#本实验中thres_kappa = 10

thres_kappa = 10
if np.max(kappa) >= thres_kappa:
    print('设计矩阵 X 存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))
else:
    print('设计矩阵 X 不存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))

设计矩阵 X 存在多重共线性，其中kappa值为：30.5051


综上所述，使用以上3种方法都判断认为存在多重共线性的，故这些特征之间存在多重共线性。

## Q3：以 BIC 为指标，对模型进行逐步回归做变量选择，判断变量选择后是否存在多重共线性。 



【1】后退法

1. 后退法，进行变量选择

In [10]:
# 后退法
col0 = list(range(X_std.shape[1]))
col1 = col0 * 1
dropped1 = True
bic_model1 = sm.OLS(Y, X_std).fit().bic
print('原始模型的bic值为：%.4f'% bic_model1)
while dropped1:
    X_std_bic1 = X_std[:, col1]
    model_bic1 = sm.OLS(Y, X_std_bic1).fit().bic
    bic1 = []
    for i in col0:
        if i not in col1:
            bic1.append(float('inf'))
            continue
        col2 = col1 * 1
        col2.remove(i)
        bic1.append(sm.OLS(Y, X_std[:, col2]).fit().bic)
    minbic = min(bic1[1:len(bic1)])
    minbic_ix = bic1.index(minbic)
    if minbic < model_bic1:
        col1.remove(minbic_ix)
    else:
        dropped1 = False
    
    if dropped1:
        print('剔除剩余变量中第 %d 列变量：' % minbic_ix, '剩余变量：', col1[1:])
        print('此时模型的bic值为：%.4f'% minbic)

原始模型的bic值为：643.5702
剔除剩余变量中第 11 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]
此时模型的bic值为：639.4795
剔除剩余变量中第 15 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14]
此时模型的bic值为：635.3994
剔除剩余变量中第 10 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14]
此时模型的bic值为：631.3236
剔除剩余变量中第 7 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 8, 9, 12, 13, 14]
此时模型的bic值为：627.4625
剔除剩余变量中第 14 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 8, 9, 12, 13]
此时模型的bic值为：623.7009
剔除剩余变量中第 8 列变量： 剩余变量： [1, 2, 3, 4, 5, 6, 9, 12, 13]
此时模型的bic值为：620.8511
剔除剩余变量中第 4 列变量： 剩余变量： [1, 2, 3, 5, 6, 9, 12, 13]
此时模型的bic值为：619.5003
剔除剩余变量中第 1 列变量： 剩余变量： [2, 3, 5, 6, 9, 12, 13]
此时模型的bic值为：618.3439
剔除剩余变量中第 3 列变量： 剩余变量： [2, 5, 6, 9, 12, 13]
此时模型的bic值为：617.9347
剔除剩余变量中第 5 列变量： 剩余变量： [2, 6, 9, 12, 13]
此时模型的bic值为：617.8083


以BIC为指标，使用后退法，将变量2、6、9、12、13纳入模型

In [11]:
# multiple linear regression
model_bic1 = sm.OLS(Y, X_std_bic1).fit()
model_bic1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     24.87
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           6.60e-13
Time:                        12:46:58   Log-Likelihood:                -296.62
No. Observations:                  60   AIC:                             605.2
Df Residuals:                      54   BIC:                             617.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.619    203.569      0.000     931.097     949.620
x1          -122.0762     46.373     -2.633      0.011    -215.048     -29.105
x2          -125.7652     40.122     -3.135      0.003    -206.205     -45.326
x3           305.7046     44.754      6.831      0.000     215.979     395.430
x4          -695.5057    233.679     -2.976      0.004   -1164.004    -227.007
x5           709.0811    225.683      3.142      0.003     256.615    1161.548
==============================================================================
Omnibus:                        0.464   Durbin-Watson:                   1.893
Prob(Omnibus):                  0.793   Jarque-Bera (JB):                0.116
Skew:                           0.086   Prob(JB):                        0.944
Kurtosis:                       3.129   Cond. No.                         70.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2. 检查经过变量选择后是否存在多重共线性

【法1】直观判定法

In [12]:
df1 = pd.DataFrame( X_std_bic1)
df1.insert(6,'6',Y)
print(df1.head())

     0         1         2         3         4         5        6
0  1.0 -0.089405  0.065713 -0.044801 -0.023850 -0.021495   921.87
1  1.0 -0.140616  0.004107 -0.122146 -0.042251 -0.035544   997.88
2  1.0 -0.063800 -0.180711 -0.161548 -0.045082 -0.046784   962.35
3  1.0  0.141043  0.019509  0.222256 -0.028096 -0.041164   982.29
4  1.0  0.013016 -0.211514  0.182854  0.007290  0.043131  1071.29


In [13]:
# 相关系数
r1 = df1.corr()
r1

,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.000000,0.116284,0.453774,0.350809,0.321014,-0.030022
2,NaN,0.116284,1.000000,-0.208774,0.286835,0.224402,-0.510988
3,NaN,0.453774,-0.208774,1.000000,-0.025865,0.018385,0.643742
4,NaN,0.350809,0.286835,-0.025865,1.000000,0.983840,-0.177242
5,NaN,0.321014,0.224402,0.018385,0.983840,1.000000,-0.077382
6,NaN,-0.030022,-0.510988,0.643742,-0.177242,-0.077382,1.000000


In [14]:
p=5

r_xy = r1.iloc[1:p+1,[p+1]].values
print('因变量和每个自变量之间的相关系数: \n', r_xy)

judge_xy = True
for i in range(p):
    if (abs(r_xy[i][0]) >= 0.5) & (model_bic1.pvalues[i+1] >= alpha):
        judge_xy = False
        print('自变量 %d 与因变量之间的简单相关系数为: %.4f, tPal: %.4f.' % (i+1, r_xy[i], model_bic1.pvalues[i+1]))
        
if judge_xy:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n存在多重共线性。')

因变量和每个自变量之间的相关系数: 
 [[-0.03002188]
 [-0.51098849]
 [ 0.64374176]
 [-0.17724211]
 [-0.07738243]]

自变量之间不存在多重共线性。


In [15]:
judge_xx = True
for (i, j) in combinations(range(1, p+1), 2):
    if(r1.iloc[i][j] >= 0.6):
        judge_xx = False
        print('变量(%d,%d)之间相关系数较大，为：%.4f'% (i, j, r1.iloc[i][j]))
        
if judge_xx:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n自变量之间存在多重共线性。')

变量(4,5)之间相关系数较大，为：0.9838

自变量之间存在多重共线性。


结论：由直观判定法认为存在多重共线性。

【法2】 方差扩大因子法

In [16]:
# 法1：
c = np.dot(X_std_bic1.T, X_std_bic1)
C = np.linalg.inv(c) 
C_list = []
for i in range(p):
    C_list.append(C[i + 1][i + 1])

# 法2：
vif = [variance_inflation_factor(X_std_bic1[:,1:p + 1], i) for i in range(p)]

print('C主对角线元素  方差扩大因子：')
for i in range(p):
    print('%d. %.4f        %.4f' % (i+1, C_list[i], vif[i]))

C主对角线元素  方差扩大因子：
1. 1.6796        1.6796
2. 1.2573        1.2573
3. 1.5644        1.5644
4. 42.6508        42.6508
5. 39.7817        39.7817


In [17]:
thres_vif = 5
flag = False
for i in range(p):
    if vif[i] >= thres_vif:
        flag = True
        print('自变量 x%d 与其余自变量之间存在多重共线性，其中VIF值为：%.4f' % (i + 1, vif[i]))

if flag==False:
    print('这种判断方式表明变量间不存在多重共线性.')
else:
    print('\n这种判断方式表明变量间存在多重共线性.')

自变量 x4 与其余自变量之间存在多重共线性，其中VIF值为：42.6508
自变量 x5 与其余自变量之间存在多重共线性，其中VIF值为：39.7817

这种判断方式表明变量间存在多重共线性.


【法3】：特征值

In [18]:
corr = np.corrcoef(X_std_bic1[:,1:p+1], rowvar = 0) 
w, v = np.linalg.eig(corr) 

kappa = []
for i in range(p):
    kappa.append(np.sqrt(max(w) / w[i]))
    print('特征值%d: %.4f, kappa%d: %.4f' %(i + 1, w[i], i + 1, kappa[i]))

特征值1: 2.2957, kappa1: 1.0000
特征值2: 1.4175, kappa2: 1.2726
特征值3: 0.8485, kappa3: 1.6449
特征值4: 0.4261, kappa4: 2.3212
特征值5: 0.0122, kappa5: 13.7428


In [19]:
thres_kappa = 10
if np.max(kappa) >= thres_kappa:
    print('设计矩阵 X 存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))
else:
    print('设计矩阵 X 不存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))

设计矩阵 X 存在多重共线性，其中kappa值为：13.7428


综上3种方法，均认为存在多重共线性，故经过后退法选择出来的特征仍然有多重共线性。

【2】前进法

1. 前进法进行变量选择

In [20]:
#前进法
col1 = list(range(X_std.shape[1]))
selected = [0]
appended1 = True
bic_model2 = sm.OLS(Y, X_std).fit().bic
print('原始模型的bic值为：%.4f'% bic_model2)
while appended1:
    if len(selected) != 1:
        X_std_bic2 = X_std[:, selected]
        model_bic2 = sm.OLS(Y, X_std_bic2).fit().bic
    else:
        model_bic2 = float('inf')
    bic2 = []
    for i in col1:
        if i in selected:
            bic2.append(float('inf'))
            continue
        col2 = selected * 1
        col2.append(i)
        bic2.append(sm.OLS(Y, X_std[:, col2]).fit().bic)
    minbic = min(bic2[1:len(bic2)])
    
    minbic_ix = bic2.index(minbic)
    if minbic < model_bic2:
        selected.append(minbic_ix)
        #col1.remove(minbic_ix)
    else:
        appended1 = False
    selected.sort(reverse=False)
    if appended1:
        print('添加第 %d 列变量：' % minbic_ix, '当前变量：', selected[1:])
        print('此时模型的bic值为：%.4f'% minbic)

原始模型的bic值为：643.5702
添加第 9 列变量： 当前变量： [9]
此时模型的bic值为：641.0009
添加第 6 列变量： 当前变量： [6, 9]
此时模型的bic值为：627.5762
添加第 2 列变量： 当前变量： [2, 6, 9]
此时模型的bic值为：620.1428
添加第 14 列变量： 当前变量： [2, 6, 9, 14]
此时模型的bic值为：616.3582
添加第 1 列变量： 当前变量： [1, 2, 6, 9, 14]
此时模型的bic值为：613.7684


以BIC为指标，使用前进法，将变量1、2、6、9、14纳入模型

In [21]:
# multiple linear regression
model_bic2 = sm.OLS(Y, X_std_bic2).fit()
model_bic2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     27.35
Date:                Sun, 24 Oct 2021   Prob (F-statistic):           1.12e-13
Time:                        12:47:06   Log-Likelihood:                -294.60
No. Observations:                  60   AIC:                             601.2
Df Residuals:                      54   BIC:                             613.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        940.3585      4.466    210.539      0.000     931.404     949.313
x1           114.1437     45.248      2.523      0.015      23.427     204.861
x2          -126.7634     40.645     -3.119      0.003    -208.252     -45.275
x3           -82.8754     42.645     -1.943      0.057    -168.373       2.622
x4           278.6269     44.895      6.206      0.000     188.618     368.636
x5           138.2354     38.537      3.587      0.001      60.973     215.498
==============================================================================
Omnibus:                        4.290   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.117   Jarque-Bera (JB):                3.443
Skew:                           0.413   Prob(JB):                        0.179
Kurtosis:                       3.834   Cond. No.                         13.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2. 检查是否存在多重共线性

【法1】直观判定法

In [22]:
df2 = pd.DataFrame(X_std_bic2)
df2.insert(6,'6',Y)
print(df2.head())

     0         1         2         3         4         5        6
0  1.0 -0.017820 -0.089405  0.065713 -0.044801  0.010748   921.87
1  1.0 -0.030859 -0.140616  0.004107 -0.122146 -0.030327   997.88
2  1.0  0.086491 -0.063800 -0.180711 -0.161548 -0.042650   962.35
3  1.0  0.125608  0.141043  0.019509  0.222256 -0.061134   982.29
4  1.0  0.073452  0.013016 -0.211514  0.182854  0.312651  1071.29


In [23]:
# 相关系数
r2 = df2.corr()
r2

,0,1,2,3,4,5,6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.000000,0.092208,-0.490425,0.413204,-0.106924,0.509500
2,NaN,0.092208,1.000000,0.116284,0.453774,-0.107810,-0.030022
3,NaN,-0.490425,0.116284,1.000000,-0.208774,-0.234346,-0.510988
4,NaN,0.413204,0.453774,-0.208774,1.000000,0.159293,0.643742
5,NaN,-0.106924,-0.107810,-0.234346,0.159293,1.000000,0.425898
6,NaN,0.509500,-0.030022,-0.510988,0.643742,0.425898,1.000000


In [24]:
p=5

r_xy = r2.iloc[1:p+1,[p+1]].values
print('因变量和每个自变量之间的相关系数: \n', r_xy)

judge_xy = True
for i in range(p):
    if (abs(r_xy[i][0]) >= 0.5) & (model_bic2.pvalues[i+1] >= alpha):
        judge_xy = False
        print('自变量 %d 与因变量之间的简单相关系数为: %.4f, tPal: %.4f.' % (i+1, r_xy[i][0], model_bic2.pvalues[i+1]))
        
if judge_xy:
    print('\n自变量之间不存在多重共线性。')
else:
    print('\n存在多重共线性。')

因变量和每个自变量之间的相关系数: 
 [[ 0.50949986]
 [-0.03002188]
 [-0.51098849]
 [ 0.64374176]
 [ 0.42589789]]
自变量 3 与因变量之间的简单相关系数为: -0.5110, tPal: 0.0572.

存在多重共线性。


In [25]:
judge_xx = True
for (i, j) in combinations(range(1, p+1), 2):
    if(r2.iloc[i][j] >= 0.6):
        judge_xx = False
        print('变量(%d,%d)之间相关系数较大，为：%.4f'% (i, j, r2.iloc[i][j]))
        
if judge_xx:
    print('\n自变量之间不存在多重共线性。')


自变量之间不存在多重共线性。


结论：方法一直观判定法认为存在多重共线性

【法2】方差扩大因子法

In [26]:
# 法1：
c = np.dot(X_std_bic2.T, X_std_bic2)
C = np.linalg.inv(c) 
C_list = []
for i in range(p):
    C_list.append(C[i + 1][i + 1])

# 法2：
vif = [variance_inflation_factor(X_std_bic2[:,1:p + 1], i) for i in range(p)]

print('C主对角线元素  方差扩大因子：')
for i in range(p):
    print('%d. %.4f        %.4f' % (i+1, C_list[i], vif[i]))

C主对角线元素  方差扩大因子：
1. 1.7105        1.7105
2. 1.3802        1.3802
3. 1.5193        1.5193
4. 1.6839        1.6839
5. 1.2408        1.2408


In [27]:
thres_vif = 5
flag = False
for i in range(p):
    if vif[i] >= thres_vif:
        flag = True
        print('自变量 x%d 与其余自变量之间存在多重共线性，其中VIF值为：%.4f' % (i + 1, vif[i]))

if flag==False:
    print('这种判断方式表明变量间不存在多重共线性.')
else:
    print('这种判断方式表明变量间存在多重共线性.')

这种判断方式表明变量间不存在多重共线性.


【法3】特征值法:

In [28]:
corr = np.corrcoef(X_std_bic2[:,1:p+1], rowvar = 0)
w, v = np.linalg.eig(corr) 

kappa = []
for i in range(p):
    kappa.append(np.sqrt(max(w) / w[i]))
    print('特征值%d: %.4f, kappa%d: %.4f' %(i + 1, w[i], i + 1, kappa[i]))

特征值1: 1.8441, kappa1: 1.0000
特征值2: 0.3201, kappa2: 2.4002
特征值3: 0.4634, kappa3: 1.9950
特征值4: 1.0445, kappa4: 1.3287
特征值5: 1.3279, kappa5: 1.1785


In [29]:
thres_kappa = 10
if np.max(kappa) >= thres_kappa:
    print('设计矩阵 X 存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))
else:
    print('设计矩阵 X 不存在多重共线性，其中kappa值为：%.4f' % np.max(kappa))

设计矩阵 X 不存在多重共线性，其中kappa值为：2.4002


以第一种直观判断法作为判断依据认为存在多重共线性，而方差扩大因子法和特征值法均认为不存在多重共线性。

综上，使用后退法进行变量选择后仍然存在多重共线性；使用前进法，用直观判定法，存在多重共线性的，当以方差扩大因子或特征值作为判断依据时，不存在多重共线性。